In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, cross_validate
from sklearn.linear_model import LinearRegression

from helper_functions import find_outlier_games

In [2]:
# Import processed data from EDA notebook
bgg = pd.read_csv('./data/bgg_processed.csv', index_col='game_id')

In [3]:
# Split off target column: `avg_rating`
y = bgg['avg_rating']
X = bgg.drop('avg_rating', axis=1).copy()

In [4]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Initial linear regression

First let's see how simple linear regression fares. There are 2000 entries in the original data, so the train/test split leaves 1600 rows in the training data. 8 splits for K-Fold cross validation should ve sufficient so each slice is 200 rows. The rows were already shuffled with the train/test split, so they don't need shuffled again.

In [5]:
# K-Fold cross validation
cv = KFold(n_splits=8)

model = LinearRegression()

scores = cross_val_score(
    model,
    X_train,
    y_train,
    scoring='neg_mean_squared_error',
    cv=cv
)

# Find RMSE
rmse = np.mean(np.sqrt(np.absolute(scores)))
rmse

20290175055.225716

Clearly, something isn't optimal. The range between the minimum score and maximum score is 2.7 points. We'd want the average RMSE to be smaller than this range, not 20 billion.

In [6]:
# Look at scores from each split
for i in scores:
    score = np.sqrt(np.absolute(i))
    print(f'Slice {list(scores).index(i)} RMSE: {score}')

Slice 0 RMSE: 0.44547528136469133
Slice 1 RMSE: 0.3831761390119257
Slice 2 RMSE: 0.3741414727970673
Slice 3 RMSE: 0.36228473775631453
Slice 4 RMSE: 0.31812747589049367
Slice 5 RMSE: 0.3616080264109322
Slice 6 RMSE: 0.39168740199763735
Slice 7 RMSE: 162321400439.16922


There is something in the 8th slice that is going wrong. Let's dig in a little to see what it is. First, we need to see what the CV is actually predicting:

In [7]:
# Generate predictions from the cross validation
preds = cross_val_predict(
    model,
    X_train,
    y_train,
    cv=cv
)

In [8]:
preds

array([7.18031593, 7.14230428, 6.73584719, ..., 7.52238108, 7.22286393,
       7.19165134])

In [9]:
print(f'Max prediction: {np.max(preds)}\nMin prediction: {np.min(preds)}')

Max prediction: 11.409906691914584
Min prediction: -2295571259637.283


It looks like the highest prediction is within reason. Ideally, we'd want it below 10, but it isn't too far off. The lowest prediction is causing a problem.

In [10]:
# Find index of smallest prediction in X_train
outlier_idx = X_train.iloc[[list(preds).index(np.min(preds))]].index[0]
outlier_idx

46213

In [11]:
# Load the names df
names = pd.read_csv('./data/bgg_names.csv', index_col='game_id')

In [12]:
# Find name of game causing the issue
names.loc[outlier_idx][0]

'Telestrations'

The model is predicting a very anomalous result for the game called "Telestrations". Is this a result that is specific for 8 k-fold slices?

In [13]:
# find_outlier_games returns {Game Name: Predicted Score} for any game
# where predicted score is -20 < score < 20
for i in range(2, 21):
    outliers = find_outlier_games(X_train, y_train, names, i)
    print(f'{i} folds - Outlier games:')
    print('\t', outliers)

2 folds - Outlier games:
	 {'Diplomacy': 12190501.043050587, 'Blood on the Clocktower': 12190501.304880919, 'Ra': 210282895078.39734, 'The Estates': 210282895077.12515, 'Wings of War: Famous Aces': 115311218752.27881, 'Paths of Glory': 115311218751.66248, 'The Grizzled': 115311218751.20435, 'No Thanks!': 210282895077.02087, 'Cosmic Encounter': 12190500.91224652, 'History of the World': 115311218751.24864, 'A Game of Thrones: The Board Game (Second Edition)': 12190501.32368377, 'Nightmare Productions': 210282895077.4625, 'Cosmic Encounter: 42nd Anniversary Edition': 12190501.28717232, 'Tiny Epic Kingdoms': 12190500.60113901, 'Citadels': -653071117.8103834, 'Arkham Horror': -653071117.7523369, 'Rhino Hero': -653071118.2375696, 'Telestrations': -118812254626.68163}
3 folds - Outlier games:
	 {'Ra': 3502737523324.3457, 'The Estates': 3502737523323.9014, 'Wings of War: Famous Aces': 241372749046.84573, 'Paths of Glory': 241372749046.48447, 'The Grizzled': 241372749045.76154, 'No Thanks!': 3

### Prediction outliers

Depending on the amount of folds used, there can be more than one game receiving extreme predictions. However, no matter how many folds used, "Telestrations" always receives an extreme prediction.

#### "Telestrations" options:
 1) Remove it from analysis
 2) Use a method, such as Principle Component Regression or Princible Least Squares Regression, to reduce the features the Linear Regression model predicts off.

PCR seems like a promising idea because there are predictions for other games that are outliers, depending on the splits used for Cross Validation. This could lead to large outliers in the test data, or worse, input data used in production.

### Removing "Telestrations"

First, let's see how the data fares without "Telestrations"

In [14]:
# Remove "Telestrations"

# Find Index
tele_idx = names[names['name']=="Telestrations"].index[0]

# Remove Game
X_train_no_tele = X_train.drop(index=tele_idx, axis=1).copy()
y_train_no_tele = y_train.drop(index=tele_idx, axis=1).copy()

In [15]:
# Linear Regression with
# K-Fold cross validation
cv = KFold(n_splits=8)

model = LinearRegression()

scores = cross_val_score(
    model,
    X_train_no_tele,
    y_train_no_tele,
    scoring='neg_mean_squared_error',
    cv=cv
)

# Find RMSE
rmse = np.mean(np.sqrt(np.absolute(scores)))
rmse

0.37248410927993547

This is a much more reasonable RMSE. How does the reduced dataset perform on different slices?

In [16]:
# find_outlier_games returns {Game Name: Predicted Score} for any game
# where predicted score is -20 < score < 20
for i in range(2, 21):
    outliers = find_outlier_games(X_train_no_tele, y_train_no_tele, names, i)
    print(f'{i} folds - Outlier games:')
    print('\t', outliers)

2 folds - Outlier games:
	 {'Diplomacy': -62362018.09583988, 'Blood on the Clocktower': -62362017.83267179, 'Ra': -975682435924.0985, 'The Estates': -975682435925.3705, 'Wings of War: Famous Aces': -524821172498.9899, 'Paths of Glory': -524821172499.6047, 'The Grizzled': -524821172500.063, 'No Thanks!': -975682435925.4734, 'Cosmic Encounter': -62362018.22522544, 'History of the World': -524821172500.01807, 'A Game of Thrones: The Board Game (Second Edition)': -62362017.81186701, 'Nightmare Productions': -975682435925.0319, 'Cosmic Encounter: 42nd Anniversary Edition': -62362017.850279115, 'Tiny Epic Kingdoms': -62362018.53601172, 'Citadels': -653071117.8103834, 'Arkham Horror': -653071117.7523369, 'Rhino Hero': -653071118.2375696}
3 folds - Outlier games:
	 {'Ra': -28203743514906.65, 'The Estates': -28203743514907.09, 'Wings of War: Famous Aces': -5596261613805.6455, 'Paths of Glory': -5596261613805.712, 'The Grizzled': -5596261613806.417, 'No Thanks!': -28203743514907.14, 'History of 

There are still a number of anomalous results with some fold selections, but the majority of fold selections produce acceptable results.

### Initial Linear regression thoughts

Linear regression by itself does not seem to be a good choice for the model. There is the possibility of predicting some very high or low predictions, depending on the training data used. There is also one game in the training data that absolutely is not predicted well. This has the potential to lead to very undesirable results in production.